In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.utils import shuffle
from joblib import dump

In [2]:
import os
import multiprocessing
n_cpu = multiprocessing.cpu_count()

In [3]:
!mkdir output

In [4]:
X = pd.read_csv("../input/features_scaled.csv")


In [5]:
Y = pd.read_csv("../input/logits.csv")

In [6]:
X.head()

,Unnamed: 0,mean,std,max,min,Rmean,Rstd,Rmax,Rmin,Imean,Istd,Imax,Imin,Rmean_last_5000,Rstd__last_5000,Rmax_last_5000,Rmin_last_5000,Rmean_last_15000,Rstd_last_15000,Rmax_last_15000,Rmin_last_15000,mean_change_abs,mean_change_rate,abs_max,abs_min,std_first_50000,std_last_50000,std_first_10000,std_last_10000,avg_first_50000,avg_last_50000,avg_first_10000,avg_last_10000,min_first_50000,min_last_50000,min_first_10000,min_last_10000,max_first_50000,max_last_50000,max_first_10000,...,q01_roll_std_100,q05_roll_std_100,q95_roll_std_100,q99_roll_std_100,av_change_abs_roll_std_100,av_change_rate_roll_std_100,abs_max_roll_std_100,ave_roll_mean_100,std_roll_mean_100,max_roll_mean_100,min_roll_mean_100,q01_roll_mean_100,q05_roll_mean_100,q95_roll_mean_100,q99_roll_mean_100,av_change_abs_roll_mean_100,av_change_rate_roll_mean_100,abs_max_roll_mean_100,ave_roll_std_1000,std_roll_std_1000,max_roll_std_1000,min_roll_std_1000,q01_roll_std_1000,q05_roll_std_1000,q95_roll_std_1000,q99_roll_std_1000,av_change_abs_roll_std_1000,av_change_rate_roll_std_1000,abs_max_roll_std_1000,ave_roll_mean_1000,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,0,1.424140,-0.170214,-0.218194,0.193218,1.199316,-0.105007,1.424140,0.156736,-0.458570,-0.169052,-0.085419,0.085419,-0.050936,-0.191893,-0.205689,0.051904,0.234333,-0.171669,-0.117546,0.156736,-1.326420,-1.569265,-0.222567,0.0,0.052067,-0.285579,0.724223,-0.192422,1.601861,0.367222,2.169234,0.502224,-0.042513,0.465851,-0.563306,0.406807,0.011907,-0.400167,0.718612,...,0.087853,0.138030,-0.252853,-0.188298,-0.125616,-1.418309,-0.185996,1.423148,-0.098604,-0.153539,0.241431,0.305730,1.053727,1.245910,0.155664,-0.487149,-1.392181,-0.153194,-0.164639,-0.187411,-0.112163,0.286995,0.102309,0.184599,-0.431605,-0.208502,-0.296462,-1.823622,-0.112163,1.421036,0.268470,-0.004742,0.178278,0.287332,0.965402,1.509153,0.885262,-0.631300,-1.832422,-0.004742
1,1,0.805716,0.004734,0.063936,-0.018037,0.078886,-0.008599,0.805716,-0.222118,0.286705,0.007212,0.042859,-0.042859,0.002525,0.004039,-0.105219,0.097303,0.421938,0.014326,0.137448,-0.222118,0.002747,-1.040206,0.036797,0.0,0.153858,-0.076987,-0.195869,-0.310470,0.648209,0.978585,0.845958,1.270320,-0.384878,-0.140002,0.197634,0.282559,0.541182,0.026977,-0.316481,...,0.053418,0.141753,0.031138,-0.043328,-0.015101,-1.107457,0.086114,0.805142,-0.046104,0.115951,0.005951,0.198170,0.697867,0.621246,0.022997,-0.137008,-1.097765,0.112852,0.087108,0.017981,-0.022923,-0.143446,0.050644,-0.151082,-0.048382,0.148106,-0.003689,-0.895105,-0.022923,0.802052,-0.141264,0.007341,-0.025387,0.622391,0.842747,0.522428,0.294357,-0.912054,-0.890022,0.007341
2,2,1.511155,0.049252,-0.086289,0.163039,0.078886,0.043800,1.511155,-0.104254,-0.540244,0.054839,0.138553,-0.138553,1.008297,0.057049,0.112694,-0.082458,-0.191007,0.057127,0.094468,-0.104254,-0.218781,0.949925,-0.101306,0.0,0.004241,0.277885,0.373967,0.681222,0.676055,2.067734,0.532791,2.128155,0.024737,-0.081995,-0.451701,-0.447401,-0.036209,0.179905,0.622003,...,1.143455,0.856220,0.666432,0.071185,0.121662,0.526623,-0.178218,1.509946,-0.005550,-0.058425,0.122544,0.204497,0.989025,1.508926,0.361297,1.081484,0.516322,-0.059295,0.350030,0.007162,-0.061793,1.038796,0.732837,0.769589,0.650231,-0.038828,0.064478,0.635232,-0.061793,1.508574,0.085078,0.099556,0.245184,0.634878,1.207106,1.530919,0.889790,0.441128,0.639209,0.099556
3,3,1.494934,0.043950,0.122560,-0.187796,0.078886,0.044798,1.494934,-0.033851,-0.417733,0.042217,0.047721,-0.047721,0.164562,0.187645,0.261908,-0.270646,-0.979372,0.060224,0.093069,-0.033851,0.002747,-0.634909,0.097427,0.0,0.020852,-0.058805,0.171820,-0.191908,1.173459,1.871230,1.693273,2.100493,0.171465,0.092027,-0.056013,0.096186,-0.187431,-0.115405,0.318376,...,0.497101,0.375361,0.124566,0.021779,-0.205746,-0.660198,0.208466,1.493770,0.056268,0.180579,

In [7]:
Y.head()

,Unnamed: 0,time_to_failure
0,0,1.430797
1,1,1.391499
2,2,1.353196
3,3,1.313798
4,4,1.274400


In [8]:
X = X.iloc[:,1:]
Y = Y.iloc[:,1:]

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from keras import backend as K
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.models import Model

Using TensorFlow backend.


In [10]:
X.shape

(4194, 154)

In [11]:
# n = X.shape[1]
n = 2
inputs = Input(shape = (154,))
x = Dense(n*400, activation = 'relu')(inputs)
x = Dense(n*500, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(n*500, activation = 'relu')(x)
x = Dense(n*600, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(n*700, activation = 'relu')(x)
x = Dense(n*800, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(n*900, activation = 'relu')(x)
x = Dense(n*1000, activation = 'relu')(x)
x = BatchNormalization()(x)
outputs = Dense(1, activation = 'linear')(x)
model = Model(inputs = inputs, outputs = outputs)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
from keras.optimizers import SGD
lr = 5e-4
model.compile(optimizer = SGD(lr), loss = "mae")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 154)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 800)               124000    
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              801000    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000)              4000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_4 (Dense)              (None, 1200)              1201200   
__________

In [13]:
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import shuffle
df = pd.concat([X.reset_index(drop=True), Y], axis=1)
df = shuffle(df)
X = df.iloc[:,:-1]
Y = df.iloc[:,-1:]

In [14]:
X.head()

,mean,std,max,min,Rmean,Rstd,Rmax,Rmin,Imean,Istd,Imax,Imin,Rmean_last_5000,Rstd__last_5000,Rmax_last_5000,Rmin_last_5000,Rmean_last_15000,Rstd_last_15000,Rmax_last_15000,Rmin_last_15000,mean_change_abs,mean_change_rate,abs_max,abs_min,std_first_50000,std_last_50000,std_first_10000,std_last_10000,avg_first_50000,avg_last_50000,avg_first_10000,avg_last_10000,min_first_50000,min_last_50000,min_first_10000,min_last_10000,max_first_50000,max_last_50000,max_first_10000,max_last_10000,...,q01_roll_std_100,q05_roll_std_100,q95_roll_std_100,q99_roll_std_100,av_change_abs_roll_std_100,av_change_rate_roll_std_100,abs_max_roll_std_100,ave_roll_mean_100,std_roll_mean_100,max_roll_mean_100,min_roll_mean_100,q01_roll_mean_100,q05_roll_mean_100,q95_roll_mean_100,q99_roll_mean_100,av_change_abs_roll_mean_100,av_change_rate_roll_mean_100,abs_max_roll_mean_100,ave_roll_std_1000,std_roll_std_1000,max_roll_std_1000,min_roll_std_1000,q01_roll_std_1000,q05_roll_std_1000,q95_roll_std_1000,q99_roll_std_1000,av_change_abs_roll_std_1000,av_change_rate_roll_std_1000,abs_max_roll_std_1000,ave_roll_mean_1000,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
3057,0.027452,-0.087698,-0.269490,0.215853,0.719132,-0.096054,0.027452,-0.167696,-0.193129,-0.090150,-0.045555,0.045555,0.071087,-0.137971,-0.115036,0.210190,0.421946,-0.079699,-0.027912,-0.167696,-0.661836,0.134670,-0.262987,0.0,-0.083572,-0.055168,-0.185810,0.092718,-0.226750,0.275670,0.355614,0.753431,0.238715,0.059801,0.177342,-0.261028,-0.297410,-0.083764,-0.054257,0.343232,...,1.004377,0.873464,0.308992,-0.085567,-0.040546,-0.677462,-0.267826,0.027420,-0.080386,-0.204755,0.156511,0.039994,-0.175608,0.128090,-0.063237,0.605292,-0.666983,-0.203755,0.118937,-0.147112,-0.235083,1.661043,1.468996,0.967368,0.213094,-0.160542,-0.005777,-0.460986,-0.235083,0.027895,0.142369,-0.130344,0.042361,-0.101836,-0.347732,0.065341,-0.104108,1.351680,-0.460627,-0.130344
261,0.115378,-0.131137,-0.166897,0.106453,0.399009,-0.126087,0.115378,0.261381,0.684865,-0.128834,-0.182753,0.182753,-0.163707,-0.060261,-0.060986,0.104549,0.117843,-0.131761,0.257551,0.261381,0.002747,0.953452,-0.165305,0.0,-0.208579,0.101055,-0.306316,-0.206048,0.305227,-0.029501,0.180397,-0.200577,0.305966,-0.178674,0.359968,0.298090,-0.400516,0.063890,-0.482095,0.014077,...,0.674615,0.314595,-0.054737,-0.142397,-0.021793,1.395713,-0.134648,0.115703,-0.105863,-0.045825,0.059198,0.160208,0.244954,-0.036295,-0.069870,-0.151013,1.382276,-0.046856,-0.045959,-0.161019,-0.164371,-0.633780,0.281579,0.314599,-0.086557,-0.148539,-0.013052,2.077555,-0.164371,0.108406,-0.281902,-0.066430,0.153030,0.241548,0.417061,-0.134180,0.015884,-1.180161,2.078594,-0.066430
2577,-0.167016,-0.302077,-0.357426,0.336570,0.078886,-0.239161,-0.167016,0.369263,0.072311,-0.302973,-0.366208,0.366208,0.302925,-0.274209,-0.355799,0.145684,-0.220981,-0.317162,-0.369607,0.369263,-0.661836,1.041195,-0.350564,0.0,-0.212034,-0.231694,-0.237863,0.097413,-0.677718,-0.019305,-1.062462,-0.176132,0.189806,0.278939,0.096175,-0.121249,-0.276789,-0.210325,-0.164667,0.239289,...,-1.154419,-1.132461,-0.619537,-0.257097,0.606997,0.409327,-0.350588,-0.166101,-0.162858,-0.268570,0.277694,0.134900,-0.046205,-0.299311,-0.215803,0.605292,0.415057,-0.266755,-0.522238,-0.294011,-0.272619,-0.426197,-0.953331,-0.971573,-0.563212,-0.249559,0.262861,-0.032062,-0.272619,-0.156390,0.023288,-0.167548,0.113896,-0.058133,-0.189001,-0.010840,-0.228628,0.860994,-0.030970,-0.167548
2994,0.186250,-0.138458,-0.163233,0.140404,-0.401299,-0.123944,0.186250,-0.045393,-0.499407,-0.143109,-0.104602,0.104602,-0.325291,-0.161837,-0.241799,0.015302,-0.650369,-0.132315,-0.101288,-0.045393,0.113511,0.401327,-0.172041,0.0,-0.097574,-0.110824,0.285064,-0.218314,-0.005786,-0.122364,-0.624094,-0.868640,0.177579,0.214487,-0.147325,0.360214,-0.187431,-0.152318,0.318376,

In [15]:
Y.head()

,time_to_failure
3057,0.807699
261,2.802197
2577,8.485600
2994,3.262299
1356,3.923996


In [16]:
trainlen = int(X.shape[0] * 0.8)
X_train = X.iloc[:trainlen,:]
X_val = X.iloc[trainlen:,:]
Y_train = Y.iloc[:trainlen,:]
Y_val = Y.iloc[trainlen:,:]

In [17]:
epochs = 12
batch_size = 32

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=1e-11)
logging = TensorBoard("output")
checkpoint = ModelCheckpoint("output/best_weights.h5", save_best_only = True, save_weights_only = True, mode = "min", monitor = "val_loss", verbose = 1)

In [18]:
model.fit(X_train,Y_train,epochs = 1000, batch_size = batch_size, validation_data = (X_val,Y_val), callbacks = [logging,checkpoint], shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 3355 samples, validate on 839 samples
Epoch 1/1000
3355/3355 [==============================] - 13s 4ms/step - loss: 5.7208 - val_loss: 4.5371

Epoch 00001: val_loss improved from inf to 4.53712, saving model to output/best_weights.h5
Epoch 2/1000
3355/3355 [==============================] - 12s 4ms/step - loss: 5.5750 - val_loss: 4.4180

Epoch 00002: val_loss improved from 4.53712 to 4.41800, saving model to output/best_weights.h5
Epoch 3/1000
3355/3355 [==============================] - 12s 4ms/step - loss: 5.4358 - val_loss: 4.3318

Epoch 00003: val_loss improved from 4.41800 to 4.33180, saving model to output/best_weights.h5
Epoch 4/1000
3355/3355 [==============================] - 13s 4ms/step - loss: 5.3152 - val_loss: 4.2083

Epoch 00004: val_loss improved from 4.33180 to 4.20827, saving model to output/best_weights.h5
Epoch 5/1000
3355/3355 [==============================] - 12s 4ms/step - loss: 5.1537 - val_loss: 4.1193


In [19]:
K.clear_session()